In [1]:
from pathlib import Path
import json
from math import sqrt
from typing import Any
from copy import deepcopy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from rich import print as rprint

In [2]:
%matplotlib qt
rng = np.random.default_rng()

In [3]:
proc_data_path_fmt = "data/experiments/analyzed_{exp_t}.json"
data = {}
for exp_t in ("tsp", "vrp", "vrpp"):
    with Path(proc_data_path_fmt.format(exp_t=exp_t)).open("r") as f:
        data[exp_t] = json.load(f)

In [4]:
criteria = tuple(data.values())[0].keys()
rprint("criteria:\n\t" + "\n\t".join(criteria))

criteria:
        by_destination_n
        by_population_size
        by_map
        by_crossover

In [33]:
criterium_translation = {
    "by_destination_n": "Rozmiar mapy (n x n)",
    "by_population_size": "Rozmiar populacji",
    "by_map": "Mapa",
}
key_translation = {
    "mean_exec_t": "czas obliczeń [s]",
    "mean_cost_improvement": "poprawa [%]",
    "mean_iters": "ilość iteracji",
}
crossover_translation = {
    "crossover_ndarray": "1-punktowy",
    "crossover_k_loci_ndarray": "k-punktowy",
    "crossover_k_loci_with_inversion_ndarray": "k-punktowy z inwersją",
    "crossover_k_loci_poisson_ndarray": "Poisson",
    "crossover_k_loci_poisson_with_inversion_ndarray": "Poisson z inwersją",
}
mutator_translation = {
    "mutate_swap": "zamiana",
    "mutate_del": "delecja",
    "mutate_insert": "insercja",
}

In [6]:
def plot_exp_stats(exp_t: str, criterium: "str", title: str):
    criterium_data = pd.DataFrame(
        {
            int(sqrt(int(dest_n))): deepcopy(vals)
            for dest_n, vals in data[exp_t][criterium].items()
        }
    ).T.sort_index()
    # to %
    criterium_data["mean_cost_improvement"] = [
        100 * x for x in criterium_data["mean_cost_improvement"]
    ]

    fig, (ax_t, ax_impr, ax_i) = plt.subplots(nrows=3, sharex=True)
    for ax, (k, k_tr) in zip([ax_t, ax_impr, ax_i], key_translation.items()):
        sns.lineplot(data=criterium_data[k], ax=ax, marker="o")
        ax.set_ylabel(k_tr)
        ax.grid()
    plt.suptitle(title)


def plot_crossovers(exp_t: str) -> pd.DataFrame:
    data_cross = deepcopy(data[exp_t]["by_crossover"])
    # to %
    for cr in data_cross:
        data_cross[cr]["mean_cost_improvement"] = (
            100 * data_cross[cr]["mean_cost_improvement"]
        )
    df = pd.DataFrame(data_cross)
    fig, (ax_t, ax_impr, ax_i) = plt.subplots(nrows=3, sharex=True)
    axs = (ax_t, ax_impr, ax_i)
    xs = list(data_cross.keys())
    for ax, k in zip(axs, ("mean_exec_t", "mean_cost_improvement", "mean_iters")):
        ys = [data_cross[cr][k] for cr in xs]
        sns.barplot(x=[crossover_translation[x] for x in xs], y=ys, ax=ax)
        ax.set_ylabel(key_translation[k])
    plt.suptitle("Operatory krzyżowania")
    return df

## Ogólne statystyki

### TSP

In [7]:
plot_exp_stats("tsp", "by_destination_n", title="Rozmiar mapy - średnie")

In [8]:
plot_exp_stats("tsp", "by_population_size", title="Rozmiar populacji - średnie")

In [9]:
df_tsp = plot_crossovers("tsp")

### VRP

In [10]:
plot_exp_stats("vrp", "by_destination_n", title="Rozmiar mapy - średnie")

In [11]:
plot_exp_stats("vrp", "by_population_size", title="Rozmiar populacji - średnie")

In [12]:
df_vrp = plot_crossovers("vrp")

### VRPP

In [13]:
plot_exp_stats("vrpp", "by_destination_n", title="Rozmiar mapy - średnie")

In [14]:
plot_exp_stats("vrpp", "by_population_size", title="Rozmiar populacji - średnie")

In [15]:
df_vrpp = plot_crossovers("vrpp")

## Konkretne eksperymenty

In [46]:
rand_exp_run_fmt = "data/experiments/runs/{exp_t}/"


def get_rand_exp_data(exp_t: str) -> dict[str, Any]:
    file_paths = {
        p
        for p in Path(rand_exp_run_fmt.format(exp_t=exp_t)).iterdir()
        if p.is_file() and p.parts and p.parts[-1].endswith(".json")
    }
    while True:
        f_path = rng.choice(tuple(file_paths))
        with f_path.open("r") as f:
            run_data = json.load(f)
            if "exception" in run_data and run_data["exception"]:
                file_paths.remove(f_path)
                continue
            break
    with Path(run_data["map_path"]).open("r") as f:
        map_data = json.load(f)
    with Path(run_data["experiment_config_path"]).open("r") as f:
        conf_data = json.load(f)
    return {
        "run": run_data,
        "map": map_data,
        "conf": conf_data,
    }


def plot_exp_data(data: dict[str, Any]) -> pd.Series:
    costs = data["run"]["costs"]
    xs_best = np.array(costs["current_best"])
    std = np.array(costs["std_dev"])
    xs_mean = np.array(costs["mean"])
    xs_mean_low = xs_mean - std
    xs_mean_high = xs_mean + std
    fig, (ax_best, ax_mean) = plt.subplots(nrows=2)
    sns.lineplot(data=xs_best, ax=ax_best, color="#0000ff", marker="o")
    sns.lineplot(data=xs_mean, ax=ax_mean, color="#ffff00", marker="o")
    sns.lineplot(data=xs_mean_high, ax=ax_mean, color="#00ff00", marker="o")
    sns.lineplot(data=xs_mean_low, ax=ax_mean, color="#00ff00", marker="o")
    ax_mean.lines[0].set_linestyle("--")
    ax_mean.lines[1].set_linestyle("--")
    ax_mean.lines[2].set_linestyle("--")
    ax_mean.fill_between(
        x=np.arange(0, len(xs_mean)), y1=xs_mean_low, y2=xs_mean_high, color="#00ff0088"
    )
    ax_mean.set_ylabel("średni koszt populacji")
    ax_best.set_ylabel("najniższy chwilowy koszt")
    plt.suptitle("Zależność kosztów od iteracji")
    conf_data = data["conf"]
    dist_mx = conf_data["dist_mx"]
    fig, ax = plt.subplots()
    sns.heatmap(data=np.array(dist_mx), cmap="magma")
    plt.title("Macierz odległości")
    n = len(dist_mx)
    p_map = {
        mutator_translation[mutator_name]: p
        for mutator_name, p in conf_data["mut_ps"].items()
    }
    ss = pd.Series(
        {
            "rozmiar populacji": str(len(conf_data["population"])),
            "rozmiar mapy": f"{n}x{n}",
            **{f"prawdopodobieństwo mutacji - {m}": p for m, p in p_map.items()},
            **(
                {
                    "prawdopodobieństwo inwersji": conf_data["crossover_kwargs"][
                        "inversion_p"
                    ]
                }
                if "inversion_p" in conf_data["crossover_kwargs"]
                else {}
            ),
        }
    )
    return ss

### TSP

In [47]:
data_rnd_tsp = get_rand_exp_data("tsp")
ss = plot_exp_data(data_rnd_tsp)
rprint(ss)

rozmiar populacji                          30
rozmiar mapy                            49x49
prawdopodobieństwo mutacji - zamiana      0.3
dtype: object

### VRP

In [48]:
data_rnd_vrp = get_rand_exp_data("vrp")
plot_exp_data(data_rnd_vrp)

rozmiar populacji                          70
rozmiar mapy                            25x25
prawdopodobieństwo mutacji - zamiana      0.3
dtype: object

### VRPP

In [49]:
data_rnd_vrpp = get_rand_exp_data("vrpp")
plot_exp_data(data_rnd_vrpp)

rozmiar populacji                           40
rozmiar mapy                             36x36
prawdopodobieństwo mutacji - zamiana       0.3
prawdopodobieństwo mutacji - insercja      0.2
prawdopodobieństwo mutacji - delecja       0.2
dtype: object